In [45]:
import numpy as np

In [46]:
d = np.array([[1, 2, 3], 
          [1, 2, 3]])
-1*d

array([[-1, -2, -3],
       [-1, -2, -3]])

In [47]:
def sigmoid(z):
  return 1 / (1 + np.exp(-1 * (z % 709)))
def sigmoid_grad(z):
  return z * (1 - z)
def leakyRelu(z):
  return np.where(z >= 0, z, z * 0.3)
def leakyRelu_grad(z):
  return np.where(z >= 0, 1, 0.3)
def softmax(z):
  e = np.exp(z)
  return e / np.sum(e)
def softmax_grad(z):
    I = np.eye(z.shape)
    return np.dot((I - softmax(z)).T, softmax(z))
def tanh(z):
  return np.tanh(z)
def tanh_grad(z):
  return 1 - np.power(tanh(z), 2)

In [48]:
Activations = {
    'sigmoid': sigmoid,
    'leakyRelu': leakyRelu,
    'softmax': softmax,
    'tanh': tanh
}
Activations_grad = {
    'sigmoid': sigmoid_grad,
    'leakyRelu': leakyRelu_grad,
    'softmax': softmax_grad,
    'tanh': tanh_grad
}

In [49]:
from sklearn.datasets import fetch_openml
import math

In [50]:
mnist = fetch_openml('mnist_784', version = 1)

In [51]:
from sklearn.model_selection import StratifiedShuffleSplit

In [52]:
sss = StratifiedShuffleSplit(n_splits = 1, train_size=10000, test_size=1000)

In [53]:
for train_index, test_index in sss.split(mnist.data, mnist.target):
  X_train = mnist.data.iloc[train_index] 
  y_train = mnist.target[train_index]
  X_test = mnist.data.iloc[test_index] 
  y_test = mnist.target[test_index]

In [54]:
def clip_image(img, clipping_size = 0):
  img = img.reshape((28, 28))
  img = np.delete(img, range(clipping_size), 0)
  img = np.delete(img, range(clipping_size), 1)
  return img.flatten()

In [55]:
def momentum(img, step_size, n_steps, clipping_size):
  img = img.reshape((28 - clipping_size, 28-clipping_size))
  momentums = []
  for i in range(n_steps):
    x = np.array(list(range(i*step_size, (i+1)*step_size)))
    for j in range(n_steps):
      pixels = img[j*step_size: (j+1)*step_size, i*step_size: (i+1) * step_size]
      y =  np.array(list(range(j*step_size, (j+1)*step_size))).reshape(step_size,1)
      area = np.sum(np.sum(pixels))
      x_c = np.sum(np.sum(x.T * pixels)) / (area + 1e-10)
      y_c = np.sum(np.sum(y * pixels)) / (area + 1e-10)
      momentums.append((x_c, y_c))
  return momentums

In [56]:
def reduction(data = None, n_momentums = 9):
  n_steps = math.floor(np.sqrt(n_momentums))
  clipping_size = 28 % n_steps
  step_size = (28 - clipping_size) // n_steps
  data = np.apply_along_axis(clipping_size = clipping_size
                      , func1d = clip_image, axis = 1, arr = data)
  momentums = np.apply_along_axis(step_size = step_size, n_steps = n_steps,
                                  clipping_size= clipping_size,
                                  func1d = momentum, axis = 1, arr = data)
  return data, momentums

In [57]:
data, momentums = reduction(np.array(X_train.copy()), n_momentums = 9)

In [58]:
from sklearn.preprocessing import LabelBinarizer
lbz = LabelBinarizer()
y_train = lbz.fit_transform(y_train)

In [59]:
X_train = momentums.reshape((10000, 18))

In [60]:
y_test.shape

(1000,)

In [61]:
X_train[0:0+1,:].shape

(1, 18)

In [236]:
class weights:
    def __init__(self, pre_layer, post_layer):
        self.pre_layer = pre_layer
        self.post_layer = post_layer
        self.weight_init = np.random.normal(size=(self.pre_layer,self.post_layer))

    def update(self, learning_rate, Dw):
        self.weight_init  = self.weight_init +  learning_rate * np.transpose(Dw)    

In [244]:
class NeuralNetwork():
    def __init__(self, layers, input_shape, nodes):
      self.layers = layers
      self.input_shape = input_shape
      self.nodes = nodes
      self.results = np.zeros(shape=(self.layers,),dtype=weights)
      self.weight = np.zeros(shape=(self.layers - 1,),dtype=weights)

    def initializing(self,train):
        data = train.copy()
        self.weight[0] = weights(data.shape[1], self.nodes)
        self.weight[self.layers - 2] = weights(self.nodes, 10)


        self.results[0] = weights(1, data.shape[1])
        self.results[self.layers - 1] = weights(1, 10)

        for i in range(1,self.layers - 2):
            self.weight[i] = weights(self.nodes, self.nodes)

        for i in range(1,self.layers - 1):
            self.results[i] = weights(1, self.nodes)


        return self.weight
  
    def forward_prop(self, train, weights):
        d_set = train.copy()

         
        #self.results[0] = weights(d_set.shape[0], d_set.shape[1])
        
        self.results[0].weight_init = d_set.copy()

        for i in range(1,self.layers-1):

            d_set = sigmoid(np.dot(d_set, weights[i-1].weight_init))

            
            #self.results[i] = weights(d_set.shape[0], d_set.shape[1])
            self.results[i].weight_init = d_set.copy()
          
        #self.results[self.layers - 1] = weights(d_set.shape[0], d_set.shape[1])
        self.results[self.layers - 1].weights_init = d_set

        return weights, self.results


    
    def back_propagation(self, results,y_train, weights, learning_rate):
        ground_truth = y_train.copy()
        weights_after = weights.copy()
        weights_before = weights.copy()
        output = results.copy()

        s_delta = np.multiply((ground_truth - output[-1].weight_init), sigmoid_grad(output[-1].weight_init))

        weights_after[-1].update(learning_rate, np.dot(np.transpose(s_delta), output[-2].weight_init))



        for i in range(output.shape[0] - 2, 0, -1):
            s_delta = np.dot(s_delta,np.transpose(weights_before[i].weight_init))
            s_delta = np.multiply(s_delta, sigmoid_grad(output[i].weight_init))

            weights_after[i-1].update(learning_rate, np.dot(np.transpose(s_delta), output[i-1].weight_init))

        return weights_after

  
    def train(self, X_train, y_train, learning_rate, num_iterations, set_weight):
      weights = set_weight.copy()
      data = X_train.copy()
      
      for epochs in range(num_iterations):
          for i in range(0,X_train.shape[0]):
              weights, outputs = self.forward_prop(data[i:i+1,:],weights)
              weights = self.back_propagation(outputs,y_train[i],weights, learning_rate)

      return weights         



    def predict(self, X, Y, weights):
        x = X.copy()

        for i in range(weights.shape[0]):
            x = sigmoid(np.dot(x,weights[i].weight_init))

        wide_train = np.zeros([Y.shape[0],Y.shape[1]])


        for m in range(0,Y.shape[0]):
            index_ = 0
            biggest = x[m,index_]
    
            for n in range(1,Y.shape[1]):
                if biggest < x[m,n]:
                    biggest = x[m,n]
                    index_ = n
            
            wide_train[m,index_] = 1    


        cnt = 0

        for r in range(x.shape[0]):
            for s in range(x.shape[1]):
                if Y[r,s] == 1 and wide_train[r,s] == 1:
                    cnt = cnt + 1

        print((cnt/Y.shape[0])*100)                

In [246]:
nn = NeuralNetwork(4,X_train,64)

In [247]:
W = nn.initializing(X_train)

In [248]:
X_train.shape

(10000, 18)

In [249]:
W = nn.train(X_train,y_train,0.5,100,W)

In [250]:
nn.predict(X_train,y_train,W)

9.87
